# Estudio de las estadísticas de un robot de limpieza reactivo

Dado:

- Habitación de MxN espacios.
- Número de agentes.
- Porcentaje de celdas inicialmente sucias.
- Tiempo máximo de ejecución.

Realiza la siguiente simulación:
- Inicializa las celdas sucias (ubicaciones aleatorias).
- Todos los agentes empiezan en la celda [1,1].
- En cada paso de tiempo:
    - Si la celda está sucia, entonces aspira.
    - Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda).
- Se ejecuta el tiempo máximo establecido.

Para un espacio de 100x100, considera los siguientes escenarios:
- Escenario 1: 1 agente, 90% de celdas sucias.
- Escenario 2. 2 agentes, 90% de celdas sucias.

Deberás resolver las siguientes preguntas:
- ¿Cuántos pasos de simulación toma limpiar todo el espacio?
- ¿Qué porcentaje de celdas sucias queda con los siguientes pasos de simulación: 100, 1000, 10000?

A continuación, determina cuál es la cantidad óptima de aspiradoras que debe de tener para realizar la limpieza en el menor tiempo posible. Considera que tenemos un máximo de 10 aspiradoras disponibles.

Desarrollar un informe con lo observado.

### Imports

In [183]:
from mesa import Agent, Model

from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd

import time
import datetime

### Vaccum Agent

In [184]:
class VaccumAgent(Agent):
    def __init__(self, id, model, x, y):
        super().__init__(id, model)
        self.position = (x, y)

    def step(self):
        if model.dirty_cells.sum() == 0:
            model.running = False

        x_pos = self.position[0]
        y_pos = self.position[1]

        # Check if the current cell is dirty
        if model.dirty_cells[x_pos][y_pos] == 1:
            model.dirty_cells[x_pos][y_pos] = 0
            # model.cleaned_cells += 1
            # model.total_dirt -= 1

        # Move to a random cell
        possible_cells = self.model.grid.get_neighborhood(self.position, moore=True, include_center = False)
        new_position = self.random.choice(possible_cells)
        cellmate = self.model.grid.get_cell_list_contents([new_position])

        if cellmate:
            return

        self.position = new_position
        self.model.grid.move_agent(self, self.position)

In [185]:
def get_vaccums(model):
    return np.asarray([agent.position for agent in model.schedule.agents])

### Vaccum Model

In [186]:
class VaccumModel(Model):
    def __init__(self, width, height, num_agents, dirty_cells_percent):
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, torus = False)
        self.datacollector = DataCollector(model_reporters={"vaccums": get_vaccums})
        self.dirty_cells = np.zeros((width, height))
        self.dirty_cells_percent = dirty_cells_percent
        self.running = True
        # self.cleaned_cells = 0
        # self.total_dirt = 0

        # Create agents and place them in the grid
        for i in range(num_agents):
            x = 1
            y = 1
            agent = VaccumAgent(i, self, x, y)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (x, y))

        # Create dirty cells in the grid
        dirty_cells = 0
        while dirty_cells < int(width * height * (dirty_cells_percent / 100)):
            x = int(np.random.rand() * width)
            y = int(np.random.rand() * height)
            if self.dirty_cells[x][y] == 0:
                self.dirty_cells[x][y] = 1
                dirty_cells += 1

    def step(self):
        if self.running == False:
            return

        self.datacollector.collect(self)
        self.schedule.step()

In [187]:
WIDTH = 100
HEIGHT = 100
DIRTY_CELLS_PERCENT = 90
ITERATIONS = [100,1000,10000]

In [188]:
def clean_cells(model, iterations, num_vaccums):
  print ("\n")
  print("Number of vaccums: ", num_vaccums)

  for iteration in iterations:
    time = tm.time()

    print("\n")
    print("Iteration: ", iteration)

    for i in range(iteration):
        model.step()

        if model.running == False:
           break

    dirty_cell = model.dirty_cells.sum()
    dirty_cell_percentage = dirty_cell / (WIDTH * HEIGHT) * 100

    print("Dirty cells: ", dirty_cell)
    print("Time: ", tm.time() - time)

In [189]:
def clean_all_cells(model, num_vaccums):
  time = tm.time()
  steps = 0

  print("\n")
  print("Number of vaccum: ", num_vaccums)

  while model.running == True:
      model.step()
      steps += 1

  print("Steps to clean all cells: ", steps)

  total_time = tm.time() - time
  print("Time: ", total_time)

  return [total_time, steps]

### Escenario 1

In [190]:
NUM_VACCUMS = 1

model = VaccumModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_CELLS_PERCENT)
clean_all_cells(model, NUM_VACCUMS)

model = VaccumModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_CELLS_PERCENT)
clean_cells(model, ITERATIONS, NUM_VACCUMS)



Number of vaccum:  1


Steps to clean all cells:  242335
Time:  9.819329738616943


Number of vaccums:  1


Iteration:  100
Dirty cells:  8965.0
Time:  0.0057027339935302734


Iteration:  1000
Dirty cells:  8706.0
Time:  0.03944802284240723


Iteration:  10000
Dirty cells:  6546.0
Time:  0.38951706886291504


### Escenario 2

In [191]:
NUM_VACCUMS = 2
model = VaccumModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_CELLS_PERCENT)
clean_all_cells(model, NUM_VACCUMS)

model = VaccumModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_CELLS_PERCENT)
clean_cells(model, ITERATIONS, NUM_VACCUMS)



Number of vaccum:  2
Steps to clean all cells:  97932
Time:  6.690618991851807


Number of vaccums:  2


Iteration:  100
Dirty cells:  8907.0
Time:  0.007847785949707031


Iteration:  1000
Dirty cells:  8428.0
Time:  0.07965993881225586


Iteration:  10000
Dirty cells:  5090.0
Time:  0.7873280048370361


### Escenario 3

In [192]:
agents = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
steps = []
times = []

for i in agents:
    NUM_VACCUMS = i
    model = VaccumModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_CELLS_PERCENT)

    [_steps, _total_time] = clean_all_cells(model, NUM_VACCUMS)
    steps.append(_steps)
    times.append(_total_time)



Number of vaccum:  1
Steps to clean all cells:  260947
Time:  10.110321760177612


Number of vaccum:  2
Steps to clean all cells:  154725
Time:  10.294326066970825


Number of vaccum:  3
Steps to clean all cells:  82554
Time:  7.6578381061553955


Number of vaccum:  4
Steps to clean all cells:  71070
Time:  8.677252292633057


Number of vaccum:  5
Steps to clean all cells:  48078
Time:  7.473476886749268


Number of vaccum:  6
Steps to clean all cells:  35473
Time:  6.460430860519409


Number of vaccum:  7
Steps to clean all cells:  34536
Time:  9.17567777633667


Number of vaccum:  8
Steps to clean all cells:  27487
Time:  8.125147104263306


Number of vaccum:  9
Steps to clean all cells:  35928
Time:  11.276005268096924


Number of vaccum:  10
Steps to clean all cells:  27449
Time:  10.47147512435913
